In [1]:
import os

In [2]:
%pwd

'/mnt/d/DeepLearning/Text_Summarizer_end-to-end_nlp_project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/mnt/d/DeepLearning/Text_Summarizer_end-to-end_nlp_project'

### Update config.yaml ✅
### Update params.yaml ✅
### Update entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

### Update the configuration manager

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import create_directories, read_yaml

In [8]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:

        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

#### Update the component

In [9]:
import os
from text_summarizer.logging.logger import logging
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/home/raghvender/Application/miniconda3/envs/text_summarization/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[ 2023-12-31 17:17:33,859] 58 datasets - INFO - config - PyTorch version 2.1.2+cu121 available.


In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encondings = self.tokenizer(example_batch['summary'], max_length=1024, truncation=True)

        return {'input_ids' : input_encodings['input_ids'],
                'attention_mask' : input_encodings['attention_mask'],
                'labels' : target_encondings['input_ids']
        }
    
    def convert(self):
        data_samsum = load_from_disk(self.config.data_path)
        data_samsum_pt = data_samsum.map(self.convert_examples_to_features, batched=True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

### Update pipeline

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[ 2023-12-31 17:17:34,040] 36 root - INFO - common - Read yaml file: config/config.yaml
[ 2023-12-31 17:17:34,044] 36 root - INFO - common - Read yaml file: params.yaml
[ 2023-12-31 17:17:34,047] 59 root - INFO - common - Created directory at: artifacts
[ 2023-12-31 17:17:34,051] 59 root - INFO - common - Created directory at: artifacts/data_transformation


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/raghvender/Application/miniconda3/envs/text_summarization/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 33723.30 examples/s]
